In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import nltk
import tensorflow as tf
import os
import string
import re
import warnings
warnings.simplefilter("ignore")
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.utils import to_categorical
data = pd.read_excel('sentence_past_present_future.xlsx')
y = data['Categorie']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
print(y[0],y[1000],y[2700])
num_classes = len(np.unique(y))
one_hot_labels = to_categorical(y, num_classes=num_classes)
y = np.array(one_hot_labels)
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import one_hot
voc_size = 10000
input = []
for x in data['Sentences']:
    val = word_tokenize(x)
    one_rpr = one_hot(str(val),n=voc_size,lower=False,split=None)
    input.append(one_rpr)
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
sent_length = 15
embedded_docs = pad_sequences(input,padding='post',maxlen=sent_length)
dim = 10
model =Sequential()
model.add(Embedding(voc_size,dim,input_length=sent_length))
model.compile('adam','mse')
model.summary()
x = model.predict(embedded_docs)
print(y.shape)
# rows = x.shape[0] 
# cols = x.shape[1] * x.shape[2]
# x = x.reshape(rows,cols)
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.25,random_state=42)
print(x_train.shape)
print(y_train.shape)

1 2 0
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 10)            100000    
                                                                 
Total params: 100000 (390.62 KB)
Trainable params: 100000 (390.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
87/87 [==============================] - 0s 930us/step
(2769, 3)
(2076, 15, 10)
(2076, 3)


In [2]:
rnn = Sequential([
     #Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length), # good for embedding
    SimpleRNN(units=128,input_shape=(15,10)),
    Dense(units=64,activation='relu'),
    Dense(units=3, activation='softmax')
])

In [3]:
rnn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [4]:
rnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               17792     
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 26243 (102.51 KB)
Trainable params: 26243 (102.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
epochs = 100
rnn.fit(x_train,y_train, epochs=epochs)

Epoch 1/100
65/65 [==============================] - 2s 4ms/step - loss: 1.0581 - accuracy: 0.4364
Epoch 2/100
65/65 [==============================] - 0s 4ms/step - loss: 1.0046 - accuracy: 0.5058
Epoch 3/100
65/65 [==============================] - 0s 4ms/step - loss: 0.9468 - accuracy: 0.5790
Epoch 4/100
65/65 [==============================] - 0s 4ms/step - loss: 0.8442 - accuracy: 0.6378
Epoch 5/100
65/65 [==============================] - 0s 4ms/step - loss: 0.8156 - accuracy: 0.6440
Epoch 6/100
65/65 [==============================] - 0s 4ms/step - loss: 0.7687 - accuracy: 0.6705
Epoch 7/100
65/65 [==============================] - 0s 4ms/step - loss: 0.7428 - accuracy: 0.6749
Epoch 8/100
65/65 [==============================] - 0s 4ms/step - loss: 0.7247 - accuracy: 0.6883
Epoch 9/100
65/65 [==============================] - 0s 4ms/step - loss: 0.6914 - accuracy: 0.6927
Epoch 10/100
65/65 [==============================] - 0s 4ms/step - loss: 0.6879 - accuracy: 0.6908
Epoch 11/

In [6]:
rnn.evaluate(x_test,y_test)

22/22 [==============================] - 0s 3ms/step - loss: 1.2800 - accuracy: 0.7128


[1.279965877532959, 0.7128427028656006]

In [7]:
def try_other_sentences(sentence):
    voc_size_val = 10000
    input_val = [sentence] 
    out_put = []
    for x in input_val:
        val = word_tokenize(x)
        one_rpr = one_hot(str(val),n=voc_size_val,lower=False,split=None)
        out_put.append(one_rpr)
    sent_length = 15
    embedded_sentence = pad_sequences(out_put,padding='post',maxlen=sent_length) #input- sentence
    value_to_pass = model.predict(embedded_sentence) 
    # rows = value_to_pass.shape[0]
    # cols = value_to_pass.shape[1] * value_to_pass.shape[2]
    # value_to_pass = value_to_pass.reshape(rows,cols) #coverting test sentence to same shape of 'x' . as we trained on.
    return value_to_pass

In [53]:
value = try_other_sentences('I was wrong about this')

1/1 [==============================] - 0s 25ms/step


In [54]:
predictions = rnn.predict(value)

1/1 [==============================] - 0s 29ms/step


In [55]:
predict_class = np.argmax(predictions,axis=1)

In [56]:
print(predictions)

[[5.0909845e-09 9.9994290e-01 5.7071495e-05]]


In [57]:
print(predict_class) #[1] index for past , [2] index for present , [0] index for future ....if o - past , 1 -present , 2-future

[1]


In [58]:
label_to_index = {"past": 1, "present": 2,"future":0}
index_to_label = {v: k for k, v in label_to_index.items()}

In [59]:
predicted_labels = [index_to_label[index] for index in predict_class]
print("\nPredicted Original Labels:", predicted_labels)


Predicted Original Labels: ['past']


In [15]:
# y[2700] , data['Categorie'][2700] 

In [16]:
# y[1000] , data['Categorie'][1000] 

In [17]:
# y[0] , data['Categorie'][0] 